<br>

# Introdução


In [ ]:
import time
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By


In [ ]:
from driver import driver
from paths import *


In [ ]:
URL = 'https://www.cati.sp.gov.br/portal/institucional/enderecos'
driver.get(URL)


<br>

## Navegação


In [ ]:
def preview_page():
    # Lista Boxes
    boxes_xpath = driver.find_element(
        By.XPATH, '//*[@id="p1"]/*[@id="list-wrapper"]')

    try:
        a = boxes_xpath.find_element(
            By.XPATH, './/*[@class="text-center"]//li[@class="prev disabled"]')
        print('Não há mais páginas anteriores')
        return True

    except Exception as e:
        # Próxima
        next_xpath = boxes_xpath.find_element(
            By.XPATH, './/*[@class="text-center"]//li[@class="prev"]//a')
        next_xpath.click()

        # Lista Boxes: Refresh DOM
        boxes_xpath = driver.find_element(
            By.XPATH, '//*[@id="p1"]/*[@id="list-wrapper"]')
        return False

In [ ]:
def go_first_page():
    """

    """
    try:
        is_first_page = False
        while not is_first_page:
            is_first_page = preview_page()
        print('Cheguei na primeira página')
        return True
        
    except Exception as e:
        print(f'Erro: {e}')
        print(f'Erro: {driver.title}')
        return False

In [ ]:
def forward_page():
    try:
        # Lista Boxes
        boxes_xpath = driver.find_element(
            By.XPATH, '//*[@id="p1"]/*[@id="list-wrapper"]')

        # Próxima
        next_xpath = boxes_xpath.find_element(
            By.XPATH, './/*[@class="text-center"]//li[@class="next"]//a')
        next_xpath.get_attribute('outerHTML')
        next_xpath.click()

        # Lista Boxes: Refresh DOM
        boxes_xpath = driver.find_element(
            By.XPATH, '//*[@id="p1"]/*[@id="list-wrapper"]')
        return True

    except Exception as e:
        print(f'Erro: {e}')
        print(f'Erro: {driver.title}')
        return False


In [ ]:
def go_to_page(page):    
    driver.get(f'https://www.cati.sp.gov.br/portal/institucional/enderecos?page={page}')
    

In [ ]:
go_to_page(5)

In [ ]:
preview_page()


In [ ]:
go_first_page()


In [ ]:
forward_page()


<br>

## Informações


In [ ]:
def get_data_from_single_page():
    """
    Monta uma tabela com todas as informações de uma única página
    No geral, são 6 unidades por página.

    """
    df = pd.DataFrame()

    try:
        # Lista Boxes
        boxes_xpath = driver.find_element(
            By.XPATH, '//*[@id="p1"]/*[@id="list-wrapper"]')

        # Lista todos os Boxes (6 por página!)
        # Para cada box, pega todas as infos
        for box in boxes_xpath.find_elements(By.XPATH, './/*[@class="col-xs-6 col-sm-4"]'):

            # Xpaths: Title
            ca = box.find_element(By.XPATH, './/h4')
            cati = box.find_element(
                By.XPATH, './/*[@class="btn-u btn-u-green outline-outward btn-link"]')
            # Xpaths: Box
            info_xpath = box.find_element(
                By.XPATH, './/ul[@class="list-unstyled pricing-v1-content"]')

            try:
                # Tenta Pegar informações
                endereco = info_xpath.find_element(
                    By.XPATH, './/*[@class="fa fa-home"]/..')
                municipio = info_xpath.find_element(
                    By.XPATH, './/*[@class="glyphicon glyphicon-map-marker"]/..')
                email = info_xpath.find_element(
                    By.XPATH, './/*[@class="fa fa-envelope"]/..')
                telefone = info_xpath.find_element(
                    By.XPATH, './/*[@class="fa fa-phone"]/..')
                responsavel = info_xpath.find_element(
                    By.XPATH, './/*[@class="glyphicon glyphicon-user"]/..')
                tecnicos = info_xpath.find_element(
                    By.XPATH, './/*[@class="icon-users"]/..')
                find_data = True

            except:
                # Exessão de Municípios sem Unidades
                find_text = 'Não existe unidade em '
                check_data = info_xpath.find_element(
                    By.XPATH, f'.//li[contains(text(), "{find_text}")]')
                mun_info = check_data.text.split('\n')[0]
                mun = mun_info.replace(find_text, '').replace('.', '')
                print(mun_info)
                find_data = False

            if find_data:
                # Parametros
                dict_box = {
                    'ca': ca.text,
                    'cati': cati.text,
                    'cati_url': cati.get_attribute('href'),
                    'endereco': endereco.text,
                    'municipio': municipio.text,
                    'email': email.text,
                    'telefone': telefone.text,
                    'responsavel': responsavel.text,
                    'tecnicos': tecnicos.text,
                    'tem_dados': find_data,
                }
            elif find_data == False:
                dict_box = {
                    'ca': f'{mun}',
                    'cati': None,
                    'cati_url': None,
                    'endereco': None,
                    'municipio': None,
                    'email': None,
                    'telefone': None,
                    'responsavel': None,
                    'tecnicos': None,
                    'tem_dados': find_data,
                }
            df_temp = pd.DataFrame([dict_box])
            df = pd.concat([df, df_temp], ignore_index=True)
        return df

    except Exception as e:
        print(f'Erro: {e}')
        print(f'Erro: {driver.title}')


<br>

## Loop


In [ ]:
n_municipio = 645
n_box_per_page = 6
int(n_municipio / n_box_per_page) + 1
n_loops = int(np.ceil(n_municipio / n_box_per_page))
n_loops


In [ ]:
# Manualmente, vi que 114 é a última página válida
n_loops = 114

In [ ]:
df = pd.DataFrame()

# 
go_to_page(1)

# Loop
for page in range(n_loops):
    # Avança Página
    if forward_page():
        # Get Data
        df_page = get_data_from_single_page()
        df = pd.concat([df, df_page], ignore_index=True)

        # Delay
        time.sleep(1)
    else:
        print('Não avançou página com sucesso!')


<br>

## Results


In [ ]:
df.to_csv(
    output_path / 'tab_cati.csv',
    index=False,
)
df.tail()


In [ ]:
driver.quit()